In [56]:
# CP.2.2.1.b2c

import numpy as np
import copy

def gaussian_elimination__section_2_1( A_): # copied and edited from colab notebook
  A = copy.deepcopy(A_)
  n = A.shape[0]
  b = np.eye(n)
  for k in range(0,n-1):
    if ( A[k,k] == 0. ): print("oops, zero pivot encountered\n"); exit()
    for i in range(k+1,n):
      mult = A[i,k] / A[k,k]
      for j in range(k+1,n):
        A[i,j] = A[i,j] - A[k,j] * mult
      A[i,k] = 0.
      b[i,k] = mult
  return A, b

b = np.array([
                 [  4., 2., 0. ],
                 [  4.,  4., 2. ],
                 [ 2.,  2., 3. ]])

c = np.array([
                 [  1., -1., 1., 2. ],
                 [  0.,  2., 1., 0. ],
                 [ 1.,  3., 4., 4. ],
                 [0.,  2., 1., -1. ]])


print(gaussian_elimination__section_2_1(b )[0], gaussian_elimination__section_2_1(b )[1])
print(gaussian_elimination__section_2_1( b )[1] )
print(gaussian_elimination__section_2_1(c )[0])
print(gaussian_elimination__section_2_1( c)[1] )

[[4. 2. 0.]
 [0. 2. 2.]
 [0. 0. 2.]] [[1.  0.  0. ]
 [1.  1.  0. ]
 [0.5 0.5 1. ]]
[[1.  0.  0. ]
 [1.  1.  0. ]
 [0.5 0.5 1. ]]
[[ 1. -1.  1.  2.]
 [ 0.  2.  1.  0.]
 [ 0.  0.  1.  2.]
 [ 0.  0.  0. -1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 2. 1. 0.]
 [0. 1. 0. 1.]]


In [57]:
# CP.2.2.2.b

import numpy as np
import copy


def gaussian_elimination__section_2_1( A_, b_ ): # copied and edited from colab notebook
  A = copy.deepcopy(A_)
  b = copy.deepcopy(b_)
  n = A.shape[0]
  c = np.eye(n)
  for k in range(0,n-1):
    if ( A[k,k] == 0. ): print("oops, zero pivot encountered\n"); break
    for i in range(k+1,n):
      mult = A[i,k] / A[k,k]
      for j in range(k+1,n):
        A[i,j] = A[i,j] - A[k,j] * mult
      b[i] = b[i] - b[k] * mult
      A[i,k] = 0.
      c[i,k] = mult
  return A, b, c

def backward_substitution( A_, b_ ): # copied from colab notebook
  A = copy.deepcopy(A_)
  b = copy.deepcopy(b_)
  n = A.shape[0]
  for i in range(n-1,-1,-1):
    for j in range(i+1,n):
      b[i] = b[i] - A[i,j] * b[j]
    b[i] = b[i] / A[i,i]
  return b

b = np.array([
                 [  4., 2., 0. ],
                 [  4.,  4., 2. ],
                 [ 2.,  2., 3. ]])

c = np.array([
                 [ 2.],
                 [  4.],
                 [ 6.]])


print(backward_substitution(gaussian_elimination__section_2_1(b, c)[0], gaussian_elimination__section_2_1(b, c)[1] ))


[[ 1.]
 [-1.]
 [ 2.]]


In [58]:
# EX.2.3.6

import numpy as np

A = np.array([
                 [  1., 2.  ],
                 [  2., 4.01]])

b = np.array([[3.],[6.01]])

x = np.array([[1.],[1.]])
print("|| b - Ax ||_oo / || b ||_oo = ",\
      f"{np.linalg.norm(b-A@x,np.infty)/np.linalg.norm(b,np.infty)}")

for xa in [ np.array([[-10.],[6.]]),\
            np.array([[-100.],[52.]]),\
            np.array([[ -600.],[301.]]),\
            np.array([[-599.],[301.]]) ]:
  relative_forward_error = np.linalg.norm( x - xa, np.inf )\
     / np.linalg.norm( x, np.inf )
  relative_backward_error = np.linalg.norm( b - A@xa, np.inf )\
    / np.linalg.norm( b, np.inf )
  error_magnification_factor = relative_forward_error\
    / relative_backward_error
  print( "xa = [ ",f"{xa[0,0]:.0f}",",",f"{xa[1,0]:.0f}","]" )
  print( "relative forward error     = ",\
    f"{relative_forward_error:8.2f}" )
  print( "relative backward error    = ",\
    f"{relative_backward_error:8.2f}" )
  print( "error magnification factor = ",\
   f"{error_magnification_factor:8.2f}" )
  print("\n")

# (e) What is the condition number of the coefficient matrix?

K = np.linalg.norm(A,np.inf)*np.linalg.norm(np.linalg.inv(A),np.inf)
print("kappa(A) = ", f"{K:6.1f}")



|| b - Ax ||_oo / || b ||_oo =  0.0
xa = [  -10 , 6 ]
relative forward error     =     11.00
relative backward error    =      0.32
error magnification factor =     33.90


xa = [  -100 , 52 ]
relative forward error     =    101.00
relative backward error    =      0.42
error magnification factor =    241.84


xa = [  -600 , 301 ]
relative forward error     =    601.00
relative backward error    =      0.17
error magnification factor =   3612.01


xa = [  -599 , 301 ]
relative forward error     =    600.00
relative backward error    =      0.50
error magnification factor =   1202.00


kappa(A) =  3612.0


In [59]:
# CP.2.3.4

# backward error is smaller with linalgsolve and slightly smaller for forward error
# in general error_mag_factor = forward error / backward error ~ Kappa(A)

import numpy as np
import copy
from math import sqrt
import time

def backward_substitution( A, b_, inplace = False ):
  if ( inplace ):
    b = b_
  else:
    b = copy.deepcopy(b_)
#
  n = A.shape[0]
  for i in range(n-1,-1,-1):
    for j in range(i+1,n):
      b[i] = b[i] - A[i,j] * b[j]
    b[i] = b[i] / A[i,i]
#
  if not inplace:
    return b
#

def lu_no_pivoting___x( A_, inplace = False ):
#
  if ( inplace ):
    A = A_
  else:
    A = copy.deepcopy(A_)
# begin main code
  n = A.shape[0]
  for k in range(0,n-1):
    if ( A[k,k] == 0. ): print("oops, zero pivot encountered\n"); break
    for i in range(k+1,n):
      A[i,k] = A[i,k] / A[k,k]
      for j in range(k+1,n):
        A[i,j] = A[i,j] - A[k,j] * A[i,k]
# end main code
  if not inplace:
    L = np.tril(A,-1)+np.eye(n)
    U = np.triu(A)
    return L, U

def forward__substitution( A, b_, inplace = False, unit = False ):
#
  import numpy as np
  import copy
#
  if ( inplace ):
    b = b_
  else:
    b = copy.deepcopy(b_)
#
  n = A.shape[0]
  for i in range(0,n,):
    for j in range(0,i):
      b[i] = b[i] - A[i,j] * b[j]
    if not unit:
      b[i] = b[i] / A[i,i]
#
  if not inplace:
    return b

n = 5
A = np.zeros( [ n, n ], dtype=float )
for i in range(0,n):
  for j in range(0,n):
    A[i,j] = sqrt( ( i - j )**2 + n / 10. )
print(A)


[[0.70710678 1.22474487 2.12132034 3.082207   4.0620192 ]
 [1.22474487 0.70710678 1.22474487 2.12132034 3.082207  ]
 [2.12132034 1.22474487 0.70710678 1.22474487 2.12132034]
 [3.082207   2.12132034 1.22474487 0.70710678 1.22474487]
 [4.0620192  3.082207   2.12132034 1.22474487 0.70710678]]


In [60]:
# CP.2.3.4 continued

%%time
for n in range(100, 600, 100):
    A = np.zeros([n, n])
    for i in range(0, n):
        for j in range(0, n):
            A[i, j] = abs(i - j) + 1.

    x = np.ones([n, 1])
    b = A @ x
    xc = np.linalg.solve(A, b)

    relative_forward_error = np.linalg.norm(x - xc, np.inf) / np.linalg.norm(x, np.inf)
    relative_backward_error = np.linalg.norm(b - A @ xc, np.inf) / np.linalg.norm(b, np.inf)
    error_magnification_factor = relative_forward_error / relative_backward_error

    print("n = ", f"{n:4d}")
    print("relative forward error = ", f"{relative_forward_error:8.2e}")
    print("relative backward error = ", f"{relative_backward_error:8.2e}")
    print("error magnification factor = ", f"{error_magnification_factor:8.2e}")
    print("kappa( A ) = | | A | | oo ∗ | | A^{−1} | | oo = ",
          f"{np.linalg.norm(A, np.infty) * np.linalg.norm(np.linalg.inv(A), np.infty):8.2e}")
    print("\n")

n =   100
relative forward error =  4.01e-12
relative backward error =  5.40e-16
error magnification factor =  7.43e+03
kappa( A ) = | | A | | oo ∗ | | A^{−1} | | oo =  1.01e+04


n =   200
relative forward error =  1.87e-11
relative backward error =  8.14e-16
error magnification factor =  2.29e+04
kappa( A ) = | | A | | oo ∗ | | A^{−1} | | oo =  4.02e+04


n =   300
relative forward error =  7.02e-11
relative backward error =  1.05e-15
error magnification factor =  6.70e+04
kappa( A ) = | | A | | oo ∗ | | A^{−1} | | oo =  9.03e+04


n =   400
relative forward error =  1.82e-10
relative backward error =  1.45e-15
error magnification factor =  1.25e+05
kappa( A ) = | | A | | oo ∗ | | A^{−1} | | oo =  1.60e+05


n =   500
relative forward error =  2.86e-10
relative backward error =  1.63e-15
error magnification factor =  1.76e+05
kappa( A ) = | | A | | oo ∗ | | A^{−1} | | oo =  2.51e+05


CPU times: user 553 ms, sys: 362 ms, total: 915 ms
Wall time: 591 ms


In [61]:
# CP.2.3.4 continued

%%time
# Loop for n in range(100, 600, 100)
for n in range(100, 600, 100):
    A = np.zeros([n, n], dtype=float)
    for i in range(0, n):
        for j in range(0, n):
            A[i, j] = abs(i - j) + 1.

    x = np.ones([n, 1], dtype=float)
    b = A @ x
    L, U = lu_no_pivoting___x(A)
    y = forward__substitution(L, b)
    xc = backward_substitution(U, y)

    relative_forward_error = np.linalg.norm(x - xc, np.inf) / np.linalg.norm(x, np.inf)
    relative_backward_error = np.linalg.norm(b - A @ xc, np.inf) / np.linalg.norm(b, np.inf)
    error_magnification_factor = relative_forward_error / relative_backward_error

    print("n = ", f"{n:4d}")
    print("relative forward error = ", f"{relative_forward_error:8.2e}")
    print("relative backward error = ", f"{relative_backward_error:8.2e}")
    print("error magnification factor = ", f"{error_magnification_factor:8.2e}")
    print("kappa( A ) = | | A | | oo ∗ | | A^{−1} | | oo = ",
          f"{np.linalg.norm(A, np.infty) * np.linalg.norm(np.linalg.inv(A), np.infty):8.2e}")
    print("\n")

n =   100
relative forward error =  5.29e-11
relative backward error =  4.23e-14
error magnification factor =  1.25e+03
kappa( A ) = | | A | | oo ∗ | | A^{−1} | | oo =  1.01e+04


n =   200
relative forward error =  5.78e-10
relative backward error =  9.19e-14
error magnification factor =  6.29e+03
kappa( A ) = | | A | | oo ∗ | | A^{−1} | | oo =  4.02e+04


n =   300
relative forward error =  3.03e-09
relative backward error =  3.49e-13
error magnification factor =  8.67e+03
kappa( A ) = | | A | | oo ∗ | | A^{−1} | | oo =  9.03e+04


n =   400
relative forward error =  4.48e-09
relative backward error =  6.40e-13
error magnification factor =  7.00e+03
kappa( A ) = | | A | | oo ∗ | | A^{−1} | | oo =  1.60e+05


n =   500
relative forward error =  9.63e-09
relative backward error =  2.01e-13
error magnification factor =  4.80e+04
kappa( A ) = | | A | | oo ∗ | | A^{−1} | | oo =  2.51e+05


CPU times: user 1min 3s, sys: 837 ms, total: 1min 3s
Wall time: 1min 3s


In [62]:
# CP.2.7.1.b

from math import sqrt
import numpy as np
import scipy
from scipy.optimize import fsolve
from scipy.optimize import root

F = lambda x : np.array( [ x[ 0 ]**2 + 4 * x[ 1 ]**2 - 4., 4 * x[ 0 ]**2 + x[ 1 ]**2 - 4 ] )

x1 = np.array( [ 2./sqrt(5),   2./sqrt(5) ] )
x2 = np.array( [-2./sqrt(5), 2./sqrt(5) ] )
x3 = np.array( [ 2./sqrt(5),   -2./sqrt(5) ] )
x4 = np.array( [ -2./sqrt(5), -2./sqrt(5) ] )

# Let us check that || F(x1) ||_oo and || F(x2) ||_oo are small
print( "|| F(x1) ||_oo =", f"{np.linalg.norm(F(x1),np.infty):.2E}" )
print( "|| F(x2) ||_oo =", f"{np.linalg.norm(F(x2),np.infty):.2E}" )
print( "|| F(x3) ||_oo =", f"{np.linalg.norm(F(x3),np.infty):.2E}" )
print( "|| F(x4) ||_oo =", f"{np.linalg.norm(F(x4),np.infty):.2E}" )



for x, x_ in zip( [ np.array([1., 1.]), np.array([-1., 1.]), np.array([1., -1.]), np.array([-1., -1.])],
                  [ x1, x2, x3, x4] ):


  x = scipy.optimize.fsolve( F, x )

  print( "\nx = [", f"{x[0]:+20.16f}", "]"\
        "     x* = [", f"{x_[0]:+20.16f}", "]")
  print( "    [", f"{x[1]:+20.16f}", "]"\
         "          [", f"{x_[1]:+20.16f}", "]")
  print( "forward  error: || x - x* ||_oo = ", f"{np.linalg.norm(x-x_,np.infty):7.2e}",\
         "\nbackward error: || F(x) ||_oo   = ", f"{np.linalg.norm(F(x),np.infty):7.2e}")

DF = lambda x : np.array([[ 2. * x[0], 8. * x[1] ],
                          [ 8. * x[0], 2. * x[1] ] ])

for x, x_ in zip( [ np.array([0.5, 0.5]), np.array([-0.5, 0.5]), np.array([0.5, -0.5]), np.array([-0.5, -0.5])],
                  [ x1, x2, x3, x4] ):

  print( "**", "[", f"{x_[0]:+20.16f}", f"{x_[1]:+20.16f}", "]", "*******", "*******" )

  print( f"{0:2d}", "[", f"{x[0]:+20.16f}", f"{x[1]:+20.16f}",\
       "]", f"{np.linalg.norm( x - x_, np.infty):4.1e}", \
      f"{np.linalg.norm( F(x), np.infty):4.1e}" )

# this is the Newton's method loop, 6 iterations should be enough
  for i in range(1,6):

#   this is the Newton's method update
    x = x - np.linalg.solve( DF(x), F(x) )

    print( f"{i:2d}", "[", f"{x[0]:+20.16f}", f"{x[1]:+20.16f}",\
           "]", f"{np.linalg.norm( x - x_, np.infty):4.1e}",\
          f"{np.linalg.norm( F(x), np.infty):4.1e}" )

  print( "\nx = [", f"{x[0]:+20.16f}", "]"\
        "     x* = [", f"{x_[0]:+20.16f}", "]")
  print( "    [", f"{x[1]:+20.16f}", "]"\
         "          [", f"{x_[1]:+20.16f}", "]")
  print( "forward  error: || x - x* ||_oo = ", f"{np.linalg.norm(x-x_,np.infty):7.2e}",\
         "\nbackward error: || F(x) ||_oo   = ", f"{np.linalg.norm(F(x),np.infty):7.2e}")
  print("\n")


|| F(x1) ||_oo = 4.44E-16
|| F(x2) ||_oo = 4.44E-16
|| F(x3) ||_oo = 4.44E-16
|| F(x4) ||_oo = 4.44E-16

x = [  +0.8944271909999160 ]     x* = [  +0.8944271909999159 ]
    [  +0.8944271909999160 ]          [  +0.8944271909999159 ]
forward  error: || x - x* ||_oo =  1.11e-16 
backward error: || F(x) ||_oo   =  8.88e-16

x = [  -0.8944271909998546 ]     x* = [  -0.8944271909999159 ]
    [  +0.8944271909999774 ]          [  +0.8944271909999159 ]
forward  error: || x - x* ||_oo =  6.15e-14 
backward error: || F(x) ||_oo   =  3.30e-13

x = [  +0.8944271909999774 ]     x* = [  +0.8944271909999159 ]
    [  -0.8944271909998546 ]          [  -0.8944271909999159 ]
forward  error: || x - x* ||_oo =  6.15e-14 
backward error: || F(x) ||_oo   =  3.30e-13

x = [  -0.8944271909999160 ]     x* = [  -0.8944271909999159 ]
    [  -0.8944271909999160 ]          [  -0.8944271909999159 ]
forward  error: || x - x* ||_oo =  1.11e-16 
backward error: || F(x) ||_oo   =  8.88e-16
** [  +0.8944271909999159  +0.89

In [63]:
# CP.2.7.1.c

F = lambda x : np.array( [ x[ 0 ]**2 - 4 * x[ 1 ]**2 - 4., (x[0] - 1) ** 2 + x[1] ** 2 - 4 ] )

x1 = np.array( [ 4 / 5 * (1 + sqrt(6)),   1/5 * (sqrt(3 + 8 * sqrt(6))) ] )
x2 = np.array( [ 4 / 5 * (1 + sqrt(6)),   -1/5 * (sqrt(3 + 8 * sqrt(6))) ] )

# Let us check that || F(x1) ||_oo and || F(x2) ||_oo are small
print( "|| F(x1) ||_oo =", f"{np.linalg.norm(F(x1),np.infty):.2E}" )
print( "|| F(x2) ||_oo =", f"{np.linalg.norm(F(x2),np.infty):.2E}" )




for x, x_ in zip( [ np.array([2., 1.]), np.array([2., -1.])],
                  [ x1, x2] ):


  x = scipy.optimize.fsolve( F, x )

  print( "\nx = [", f"{x[0]:+20.16f}", "]"\
        "     x* = [", f"{x_[0]:+20.16f}", "]")
  print( "    [", f"{x[1]:+20.16f}", "]"\
         "          [", f"{x_[1]:+20.16f}", "]")
  print( "forward  error: || x - x* ||_oo = ", f"{np.linalg.norm(x-x_,np.infty):7.2e}",\
         "\nbackward error: || F(x) ||_oo   = ", f"{np.linalg.norm(F(x),np.infty):7.2e}")

DF = lambda x : np.array([[ 2. * x[0], -8. * x[1] ],
                          [ 2. * x[0] - 2, 2. * x[1] ] ])

for x, x_ in zip( [ np.array([2., 1.]), np.array([2., -1.])],
                  [ x1, x2] ):

  print( "**", "[", f"{x_[0]:+20.16f}", f"{x_[1]:+20.16f}", "]", "*******", "*******" )

  print( f"{0:2d}", "[", f"{x[0]:+20.16f}", f"{x[1]:+20.16f}",\
       "]", f"{np.linalg.norm( x - x_, np.infty):4.1e}", \
      f"{np.linalg.norm( F(x), np.infty):4.1e}" )

# this is the Newton's method loop, 6 iterations should be enough
  for i in range(1,6):

#   this is the Newton's method update
    x = x - np.linalg.solve( DF(x), F(x) )

    print( f"{i:2d}", "[", f"{x[0]:+20.16f}", f"{x[1]:+20.16f}",\
           "]", f"{np.linalg.norm( x - x_, np.infty):4.1e}",\
          f"{np.linalg.norm( F(x), np.infty):4.1e}" )

  print( "\nx = [", f"{x[0]:+20.16f}", "]"\
        "     x* = [", f"{x_[0]:+20.16f}", "]")
  print( "    [", f"{x[1]:+20.16f}", "]"\
         "          [", f"{x_[1]:+20.16f}", "]")
  print( "forward  error: || x - x* ||_oo = ", f"{np.linalg.norm(x-x_,np.infty):7.2e}",\
         "\nbackward error: || F(x) ||_oo   = ", f"{np.linalg.norm(F(x),np.infty):7.2e}")
  print("\n")


|| F(x1) ||_oo = 8.88E-16
|| F(x2) ||_oo = 8.88E-16

x = [  +2.7595917942265427 ]     x* = [  +2.7595917942265427 ]
    [  +0.9507032753128691 ]          [  +0.9507032753128691 ]
forward  error: || x - x* ||_oo =  0.00e+00 
backward error: || F(x) ||_oo   =  8.88e-16

x = [  +2.7595917942265427 ]     x* = [  +2.7595917942265427 ]
    [  -0.9507032753128690 ]          [  -0.9507032753128691 ]
forward  error: || x - x* ||_oo =  1.11e-16 
backward error: || F(x) ||_oo   =  1.78e-15
** [  +2.7595917942265427  +0.9507032753128691 ] ******* *******
 0 [  +2.0000000000000000  +1.0000000000000000 ] 7.6e-01 4.0e+00
 1 [  +3.0000000000000000  +1.0000000000000000 ] 2.4e-01 1.0e+00
 2 [  +2.7727272727272725  +0.9545454545454546 ] 1.3e-02 5.4e-02
 3 [  +2.7596355257645580  +0.9507201707662536 ] 4.4e-05 1.9e-04
 4 [  +2.7595917947145026  +0.9507032755656462 ] 4.9e-10 2.2e-09
 5 [  +2.7595917942265422  +0.9507032753128690 ] 4.4e-16 1.3e-15

x = [  +2.7595917942265422 ]     x* = [  +2.7595917942265427

In [64]:
# CP.2.7.4

F = lambda x: np.array([
    2 * x[0]**2 - 4 * x[0] + x[1]**2 + 3 * x[2]**2 + 6 * x[2] + 2.,
    x[0]**2 + x[1]**2 - 2 * x[1] + 2 * x[2]**2 - 5,
    3 * x[0]**2 - 12 * x[0] + x[1]**2 + 3 * x[2]**2 + 8
])

x1 = np.array( [2, 1, -1] )
x2 = np.array( [ 1.0960178410014330, -1.1592471842154839, -0.2611479367011116 ] )

# Let us check that || F(x1) ||_oo and || F(x2) ||_oo are small
print( "|| F(x1) ||_oo =", f"{np.linalg.norm(F(x1),np.infty):.2E}" )
print( "|| F(x2) ||_oo =", f"{np.linalg.norm(F(x2),np.infty):.2E}" )




for x in [ x1, x2 ]:

  print( "\nx = [", f"{x[0]:+20.16f}", "]")
  print( "    [", f"{x[1]:+20.16f}", "]")
  print( "    [", f"{x[2]:+20.16f}", "]")
  print( "backward error: || F(x) ||_oo   = ", f"{np.linalg.norm(F(x),np.infty):7.2e}")

def DF( x ):
  return np.array([[ 4 * x[0] - 4, 2 * x[1], 6 * x[2] + 6 ],
                   [ 2 * x[0], 2 * x[1] - 2, 4 * x[2] ],
                   [ 6 * x[0] - 12, 2 * x[1], 6 * x[2] ]])

for x, x_ in zip( [ np.array( [ 2., 1., 0. ] ), np.array( [ 1., -1., 0. ] ) ],
                  [ x1, x2 ] ):

  print( f"{0:2d}", "[", f"{x[0]:+20.16f}", f"{x[1]:+20.16f}", f"{x[2]:+20.16f}",\
       "]", f"{np.linalg.norm( x - x_, np.infty):4.1e}", \
      f"{np.linalg.norm( F(x), np.infty):4.1e}" )

# this is the Newton's method loop, 10 iterations should be enough
  for i in range(1,10):

#   this is the Newton's method update
    x = x - np.linalg.solve( DF(x), F(x) )

    print( f"{i:2d}", "[", f"{x[0]:+20.16f}", f"{x[1]:+20.16f}", f"{x[2]:+20.16f}",\
           "]", f"{np.linalg.norm( x - x_, np.infty):4.1e}",\
          f"{np.linalg.norm( F(x), np.infty):4.1e}" )

  print( "\nx = [", f"{x[0]:+20.16f}", "]"\
        "     x* = [", f"{x_[0]:+20.16f}", "]")
  print( "    [", f"{x[1]:+20.16f}", "]"\
         "          [", f"{x_[1]:+20.16f}", "]")
  print( "    [", f"{x[2]:+20.16f}", "]"\
         "          [", f"{x_[2]:+20.16f}", "]")
  print( "forward  error: || x - x* ||_oo = ", f"{np.linalg.norm(x-x_,np.infty):7.2e}",\
         "\nbackward error: || F(x) ||_oo   = ", f"{np.linalg.norm(F(x),np.infty):7.2e}")
  print("\n")

|| F(x1) ||_oo = 0.00E+00
|| F(x2) ||_oo = 2.24E-11

x = [  +2.0000000000000000 ]
    [  +1.0000000000000000 ]
    [  -1.0000000000000000 ]
backward error: || F(x) ||_oo   =  0.00e+00

x = [  +1.0960178410014330 ]
    [  -1.1592471842154839 ]
    [  -0.2611479367011116 ]
backward error: || F(x) ||_oo   =  2.24e-11
 0 [  +2.0000000000000000  +1.0000000000000000  +0.0000000000000000 ] 1.0e+00 3.0e+00
 1 [  +2.5000000000000000  +2.5000000000000000  -1.3333333333333333 ] 1.5e+00 8.3e+00
 2 [  +2.2019230769230766  +1.3173076923076927  -1.1426282051282051 ] 3.2e-01 1.8e+00
 3 [  +2.0160176799788596  +1.0577766769608021  -1.0163958284094203 ] 5.8e-02 2.2e-01
 4 [  +2.0005703194848459  +1.0010865322265479  -1.0004199289368834 ] 1.1e-03 4.7e-03
 5 [  +2.0000002459552815  +1.0000006868387383  -1.0000002181341494 ] 6.9e-07 2.7e-06
 6 [  +2.0000000000000879  +1.0000000000001918  -1.0000000000000691 ] 1.9e-13 8.0e-13
 7 [  +2.0000000000000000  +1.0000000000000000  -1.0000000000000000 ] 0.0e+00 0.0e

In [65]:
# CP.2.7.5.b

F = lambda x : np.array( [
                          ( x[0] - 1. )**2 + ( x[1] + 2. )**2  + ( x[2]      )**2  - 25.,
                          ( x[0] + 2. )**2 + ( x[1]  -  2.  )**2  + ( x[2] + 1. )**2  - 25.,
                          ( x[0] - 4. )**2 + ( x[1] + 2. )**2  + ( x[2] - 3. )**2  - 25. ] )

x1 = np.array( [ 17/9., 22/9., 19/9. ] )
x2 = np.array( [ 1., 2., 3. ] )

print( "|| F(x1) ||_oo =", f"{np.linalg.norm(F(x1),np.infty):.2E}" )
print( "|| F(x2) ||_oo =", f"{np.linalg.norm(F(x2),np.infty):.2E}" )

x = np.array( [ 0., 0., 0. ] )

x = scipy.optimize.fsolve( F, x )

print( "\nx = [", f"{x[0]:+20.16f}", "]")
print( "    [", f"{x[1]:+20.16f}", "]")
print( "    [", f"{x[2]:+20.16f}", "]")
print( "backward error: || F(x) ||_oo   = ", f"{np.linalg.norm(F(x),np.infty):7.2e}")

x1 = x

x = np.array( [ -2.,- 2., -2. ] )

x = scipy.optimize.fsolve( F, x )

print( "\nx = [", f"{x[0]:+20.16f}", "]")
print( "    [", f"{x[1]:+20.16f}", "]")
print( "    [", f"{x[2]:+20.16f}", "]")
print( "backward error: || F(x) ||_oo   = ", f"{np.linalg.norm(F(x),np.infty):7.2e}")

x2 = x

DF = lambda x : np.array([
  [ 2. * ( x[0] - 1. ), 2. * ( x[1] + 2. ), 2. * x[2] ],
  [ 2. * ( x[0] + 2. ), 2. * (x[1] - 2), 2. * ( x[2] + 1. ) ],
  [ 2. * (x[0] - 4), 2. * ( x[1] + 2. ), 2. * ( x[2] - 3. ) ] ] )

for x, x_ in zip( [ np.array( [ 2, 2, 2 ] ), np.array( [ 0.5, 1.5, 2.5 ] ) ],
                  [ x1, x2 ] ):

  print( f"{0:2d}", "[", f"{x[0]:+18.14f}", f"{x[1]:+18.14f}",\
         f"{x[2]:+18.14f}",\
       "]", f"{np.linalg.norm( x - x_, np.infty):4.1e}", \
      f"{np.linalg.norm( F(x), np.infty):4.1e}" )

# this is the Newton's method loop, 8 iterations should be enough
  for i in range(1,8):

#   this is the Newton's method update
    x = x - np.linalg.solve( DF(x), F(x) )

    print( f"{i:2d}", "[", f"{x[0]:+18.14f}",\
           f"{x[1]:+18.14f}", f"{x[2]:+18.14f}",\
           "]", f"{np.linalg.norm( x - x_, np.infty):4.1e}",\
          f"{np.linalg.norm( F(x), np.infty):4.1e}" )

  print( "\nx = [", f"{x[0]:+18.14f}", "]"\
        "     x* = [", f"{x_[0]:+18.14f}", "]")
  print( "    [", f"{x[1]:+18.14f}", "]"\
         "          [", f"{x_[1]:+18.14f}", "]")
  print( "    [", f"{x[2]:+18.14f}", "]"\
         "          [", f"{x_[2]:+18.14f}", "]")
  print( "forward  error: || x - x* ||_oo = ",\
 f"{np.linalg.norm(x-x_,np.infty):7.2e}",\
         "\nbackward error: || F(x) ||_oo   = ",\
 f"{np.linalg.norm(F(x),np.infty):7.2e}")
  print("\n")

|| F(x1) ||_oo = 3.55E-15
|| F(x2) ||_oo = 0.00E+00

x = [  +1.8888888888888720 ]
    [  +2.4444444444444358 ]
    [  +2.1111111111111289 ]
backward error: || F(x) ||_oo   =  3.55e-14

x = [  +1.0000000000000009 ]
    [  +2.0000000000000009 ]
    [  +2.9999999999999991 ]
backward error: || F(x) ||_oo   =  0.00e+00
 0 [  +2.00000000000000  +2.00000000000000  +2.00000000000000 ] 4.4e-01 4.0e+00
 1 [  +2.00000000000000  +2.50000000000000  +2.00000000000000 ] 1.1e-01 2.5e-01
 2 [  +1.90000000000000  +2.45000000000000  +2.10000000000000 ] 1.1e-02 2.3e-02
 3 [  +1.88902439024390  +2.44451219512195  +2.11097560975610 ] 1.4e-04 2.7e-04
 4 [  +1.88888890953829  +2.44444445476914  +2.11111109046171 ] 2.1e-08 4.1e-08
 5 [  +1.88888888888889  +2.44444444444445  +2.11111111111111 ] 2.0e-14 3.6e-15
 6 [  +1.88888888888889  +2.44444444444444  +2.11111111111111 ] 1.7e-14 3.6e-15
 7 [  +1.88888888888889  +2.44444444444444  +2.11111111111111 ] 1.7e-14 3.6e-15

x = [  +1.88888888888889 ]     x* = [  +1.8

In [66]:
# CP.2.7.7.b2c
# b : (+0.8944271909999159 , +0.8944271909999159), 11 steps to backward error of 4.44e-16 and forward error of 0.00e+00

F = lambda x : np.array( [ x[ 0 ]**2 + 4 * x[ 1 ]**2 - 4., 4 * x[ 0 ]**2 + x[ 1 ]**2 - 4 ] )

x1 = np.array( [ 2./sqrt(5),   2./sqrt(5) ] )
x2 = np.array( [-2./sqrt(5), 2./sqrt(5) ] )
x3 = np.array( [ 2./sqrt(5),   -2./sqrt(5) ] )
x4 = np.array( [ -2./sqrt(5), -2./sqrt(5) ] )

# Let us check that || F(x1) ||_oo and || F(x2) ||_oo are small
print( "|| F(x1) ||_oo =", f"{np.linalg.norm(F(x1),np.infty):.2E}" )
print( "|| F(x2) ||_oo =", f"{np.linalg.norm(F(x2),np.infty):.2E}" )
print( "|| F(x3) ||_oo =", f"{np.linalg.norm(F(x3),np.infty):.2E}" )
print( "|| F(x4) ||_oo =", f"{np.linalg.norm(F(x4),np.infty):.2E}" )

x = np.array( [ 1., 1. ] )
A = np.eye(2,2)

# for the forward error decide with starting with x=[1,1] will converge to x1 or x2
# here we see this is x1
x_ = x1

print( f"{0:2d}", "[", f"{x[0]:+20.16f}", f"{x[1]:+20.16f}",\
       "]", f"{np.linalg.norm( x - x_, np.infty):4.1e}", \
      f"{np.linalg.norm( F(x), np.infty):4.1e}" )

for i in range(1,12):

  x0 = x
  x = x - np.linalg.solve( A, F(x) )
  Delta = ( F(x) - F(x0) )
  delta = x - x0
  A = A + np.outer ( ( Delta - ( A @ delta ) ), delta / (delta.T @ delta) )

  print( f"{i:2d}", "[", f"{x[0]:+20.16f}", f"{x[1]:+20.16f}",\
           "]", f"{np.linalg.norm( x - x_, np.infty):4.1e}",\
          f"{np.linalg.norm( F(x), np.infty):4.1e}" )

print( "\nx = [", f"{x[0]:+20.16f}", "]"\
        "     x* = [", f"{x_[0]:+20.16f}", "]")
print( "    [", f"{x[1]:+20.16f}", "]"\
         "          [", f"{x_[1]:+20.16f}", "]")
print( "forward  error: || x - x* ||_oo = ", f"{np.linalg.norm(x-x_,np.infty):7.2e}",\
         "\nbackward error: || F(x) ||_oo   = ", f"{np.linalg.norm(F(x),np.infty):7.2e}")
print("\n")


|| F(x1) ||_oo = 4.44E-16
|| F(x2) ||_oo = 4.44E-16
|| F(x3) ||_oo = 4.44E-16
|| F(x4) ||_oo = 4.44E-16
 0 [  +1.0000000000000000  +1.0000000000000000 ] 1.1e-01 1.0e+00
 1 [  +0.0000000000000000  +0.0000000000000000 ] 8.9e-01 4.0e+00
 2 [  +0.7999999999999999  +0.8000000000000000 ] 9.4e-02 8.0e-01
 3 [  +0.9999999999999998  +1.0000000000000002 ] 1.1e-01 1.0e+00
 4 [  +0.8888888888888878  +0.8888888888888899 ] 5.5e-03 4.9e-02
 5 [  +0.8941176470588171  +0.8941176470588299 ] 3.1e-04 2.8e-03
 6 [  +0.8944281524926259  +0.8944281524927115 ] 9.6e-07 8.6e-06
 7 [  +0.8944271908332385  +0.8944271908337819 ] 1.7e-10 1.5e-09
 8 [  +0.8944271909981865  +0.8944271910016451 ] 1.7e-12 9.3e-12
 9 [  +0.8944271909889014  +0.8944271910109304 ] 1.1e-11 5.9e-11
10 [  +0.8944271909999159  +0.8944271909999159 ] 0.0e+00 4.4e-16
11 [  +0.8944271909999159  +0.8944271909999159 ] 0.0e+00 4.4e-16

x = [  +0.8944271909999159 ]     x* = [  +0.8944271909999159 ]
    [  +0.8944271909999159 ]          [  +0.89442719

<ipython-input-66-798aca2363e6>:34: RuntimeWarning: invalid value encountered in divide
  A = A + np.outer ( ( Delta - ( A @ delta ) ), delta / (delta.T @ delta) )


In [67]:
# CP.2.7.7.b2c
# c : (+2.7595917942265427 , -0.9507032753128698), 50 steps to backward error of 4.44e-15 and forward error of 6.66e-16
# really bounced around at the beginning and even near the ending iterations resulting in slight discrepancies towards the end

F = lambda x : np.array( [ x[0]**2 - 4 * x[1]**2 - 4., (x[0] - 1) ** 2 + x[1] ** 2 - 4 ] )

x1 = np.array( [ 4 / 5 * (1 + sqrt(6)),   1/5 * (sqrt(3 + 8 * sqrt(6))) ] )
x2 = np.array( [ 4 / 5 * (1 + sqrt(6)),   -1/5 * (sqrt(3 + 8 * sqrt(6))) ] )

# Let us check that || F(x1) ||_oo and || F(x2) ||_oo are small
print( "|| F(x1) ||_oo =", f"{np.linalg.norm(F(x1),np.infty):.2E}" )
print( "|| F(x2) ||_oo =", f"{np.linalg.norm(F(x2),np.infty):.2E}" )

x = np.array( [ 1., 1. ] )
A = np.eye(2,2)

# for the forward error decide with starting with x=[1,1] will converge to x1 or x2
# here we see this is x2
x_ = x2

print( f"{0:2d}", "[", f"{x[0]:+20.16f}", f"{x[1]:+20.16f}",\
       "]", f"{np.linalg.norm( x - x_, np.infty):4.1e}", \
      f"{np.linalg.norm( F(x), np.infty):4.1e}" )

for i in range(1,50):

  x0 = x
  x = x - np.linalg.solve( A, F(x) )
  Delta = ( F(x) - F(x0) )
  delta = x - x0
  A = A + np.outer ( ( Delta - ( A @ delta ) ), delta / (delta.T @ delta) )

  print( f"{i:2d}", "[", f"{x[0]:+20.16f}", f"{x[1]:+20.16f}",\
           "]", f"{np.linalg.norm( x - x_, np.infty):4.1e}",\
          f"{np.linalg.norm( F(x), np.infty):4.1e}" )

print( "\nx = [", f"{x[0]:+20.16f}", "]"\
        "     x* = [", f"{x_[0]:+20.16f}", "]")
print( "    [", f"{x[1]:+20.16f}", "]"\
         "          [", f"{x_[1]:+20.16f}", "]")
print( "forward  error: || x - x* ||_oo = ", f"{np.linalg.norm(x-x_,np.infty):7.2e}",\
         "\nbackward error: || F(x) ||_oo   = ", f"{np.linalg.norm(F(x),np.infty):7.2e}")
print("\n")

|| F(x1) ||_oo = 8.88E-16
|| F(x2) ||_oo = 8.88E-16
 0 [  +1.0000000000000000  +1.0000000000000000 ] 2.0e+00 7.0e+00
 1 [  +8.0000000000000000  +4.0000000000000000 ] 5.2e+00 6.1e+01
 2 [  +9.0892018779342720 -12.6103286384976556 ] 1.2e+01 5.6e+02
 3 [  +0.2916638593782324  +3.7318816068898606 ] 4.7e+00 6.0e+01
 4 [  +1.0972908318750931  +5.6983105119749977 ] 6.6e+00 1.3e+02
 5 [  +0.0775153594732114  +1.7414997185989129 ] 2.7e+00 1.6e+01
 6 [  +0.3740092015760767  +0.7701322332852983 ] 2.4e+00 6.2e+00
 7 [  +2.2549401327462721  -2.1988498971521659 ] 1.2e+00 1.8e+01
 8 [ -104.9626363441826413 +217.1421389146939589 ] 2.2e+02 1.8e+05
 9 [  +3.0997663277804293  -1.8562303885391884 ] 9.1e-01 8.2e+00
10 [  +2.8335876499036727  -1.9850625420521057 ] 1.0e+00 1.2e+01
11 [  +2.9256371659518430  -1.9566792606668006 ] 1.0e+00 1.1e+01
12 [  +2.9320532029143855  -1.9703616045585477 ] 1.0e+00 1.1e+01
13 [  +3.1913218182129950  -3.1834306474408374 ] 2.2e+00 3.4e+01
14 [  +2.8114975033177951  -1.533535

In [68]:
# CP.2.7.8.b2c
# b : (+0.8944271909999159 , +0.8944271909999159), 11 steps to backward error of 4.44e-16 and forward error of 0.00e+00

F = lambda x : np.array( [ x[ 0 ]**2 + 4 * x[ 1 ]**2 - 4., 4 * x[ 0 ]**2 + x[ 1 ]**2 - 4 ] )

x1 = np.array( [ 2./sqrt(5),   2./sqrt(5) ] )
x2 = np.array( [-2./sqrt(5), 2./sqrt(5) ] )
x3 = np.array( [ 2./sqrt(5),   -2./sqrt(5) ] )
x4 = np.array( [ -2./sqrt(5), -2./sqrt(5) ] )

# Let us check that || F(x1) ||_oo and || F(x2) ||_oo are small
print( "|| F(x1) ||_oo =", f"{np.linalg.norm(F(x1),np.infty):.2E}" )
print( "|| F(x2) ||_oo =", f"{np.linalg.norm(F(x2),np.infty):.2E}" )
print( "|| F(x3) ||_oo =", f"{np.linalg.norm(F(x3),np.infty):.2E}" )
print( "|| F(x4) ||_oo =", f"{np.linalg.norm(F(x4),np.infty):.2E}" )

x = np.array( [ 1., 1. ] )
B = np.eye(2,2)

# for the forward error decide with starting with x=[1,1] will converge to x1 or x2
# here we see this is x1
x_ = x1

print( f"{0:2d}", "[", f"{x[0]:+20.16f}", f"{x[1]:+20.16f}",\
       "]", f"{np.linalg.norm( x - x_, np.infty):4.1e}", \
      f"{np.linalg.norm( F(x), np.infty):4.1e}" )

for i in range(1,12):

  x0 = x
  x = x - B @ F(x)
  Delta = ( F(x) - F(x0) )
  delta = x - x0
  B = B + np.outer ( ( delta - ( B @ Delta ) ), ( delta.T @ B ) / (delta.T @ ( B @ Delta) ) )

  print( f"{i:2d}", "[", f"{x[0]:+20.16f}", f"{x[1]:+20.16f}",\
           "]", f"{np.linalg.norm( x - x_, np.infty):4.1e}",\
          f"{np.linalg.norm( F(x), np.infty):4.1e}" )

print( "\nx = [", f"{x[0]:+20.16f}", "]"\
        "     x* = [", f"{x_[0]:+20.16f}", "]")
print( "    [", f"{x[1]:+20.16f}", "]"\
         "          [", f"{x_[1]:+20.16f}", "]")
print( "forward  error: || x - x* ||_oo = ", f"{np.linalg.norm(x-x_,np.infty):7.2e}",\
         "\nbackward error: || F(x) ||_oo   = ", f"{np.linalg.norm(F(x),np.infty):7.2e}")
print("\n")

|| F(x1) ||_oo = 4.44E-16
|| F(x2) ||_oo = 4.44E-16
|| F(x3) ||_oo = 4.44E-16
|| F(x4) ||_oo = 4.44E-16
 0 [  +1.0000000000000000  +1.0000000000000000 ] 1.1e-01 1.0e+00
 1 [  +0.0000000000000000  +0.0000000000000000 ] 8.9e-01 4.0e+00
 2 [  +0.7999999999999998  +0.7999999999999998 ] 9.4e-02 8.0e-01
 3 [  +1.0000000000000002  +1.0000000000000002 ] 1.1e-01 1.0e+00
 4 [  +0.8888888888888888  +0.8888888888888890 ] 5.5e-03 4.9e-02
 5 [  +0.8941176470588232  +0.8941176470588238 ] 3.1e-04 2.8e-03
 6 [  +0.8944281524926667  +0.8944281524926705 ] 9.6e-07 8.6e-06
 7 [  +0.8944271908334980  +0.8944271908335222 ] 1.7e-10 1.5e-09
 8 [  +0.8944271909998386  +0.8944271909999929 ] 7.7e-14 4.2e-13
 9 [  +0.8944271909994246  +0.8944271910004071 ] 4.9e-13 2.6e-12
10 [  +0.8944271909999159  +0.8944271909999159 ] 0.0e+00 4.4e-16
11 [  +0.8944271909999159  +0.8944271909999159 ] 0.0e+00 4.4e-16

x = [  +0.8944271909999159 ]     x* = [  +0.8944271909999159 ]
    [  +0.8944271909999159 ]          [  +0.89442719

<ipython-input-68-5e12ddcb915f>:34: RuntimeWarning: invalid value encountered in divide
  B = B + np.outer ( ( delta - ( B @ Delta ) ), ( delta.T @ B ) / (delta.T @ ( B @ Delta) ) )


In [69]:
# CP.2.7.8.b2c
# c : (+2.7595917942265427 , -0.9507032753128691), 40 steps to backward error of 8.9e-16 and forward error of 0.0e+00

F = lambda x : np.array( [ x[0]**2 - 4 * x[1]**2 - 4., (x[0] - 1) ** 2 + x[1] ** 2 - 4 ] )

x1 = np.array( [ 4 / 5 * (1 + sqrt(6)),   1/5 * (sqrt(3 + 8 * sqrt(6))) ] )
x2 = np.array( [ 4 / 5 * (1 + sqrt(6)),   -1/5 * (sqrt(3 + 8 * sqrt(6))) ] )

# Let us check that || F(x1) ||_oo and || F(x2) ||_oo are small
print( "|| F(x1) ||_oo =", f"{np.linalg.norm(F(x1),np.infty):.2E}" )
print( "|| F(x2) ||_oo =", f"{np.linalg.norm(F(x2),np.infty):.2E}" )

x = np.array( [ 1., 1. ] )
B = np.eye(2,2)

# for the forward error decide with starting with x=[1,1] will converge to x1 or x2
# here we see this is x2
x_ = x2

print( f"{0:2d}", "[", f"{x[0]:+20.16f}", f"{x[1]:+20.16f}",\
       "]", f"{np.linalg.norm( x - x_, np.infty):4.1e}", \
      f"{np.linalg.norm( F(x), np.infty):4.1e}" )

for i in range(1,40):

  x0 = x
  x = x - B @ F(x)
  Delta = ( F(x) - F(x0) )
  delta = x - x0
  B = B + np.outer ( ( delta - ( B @ Delta ) ), ( delta.T @ B ) / (delta.T @ ( B @ Delta) ) )

  print( f"{i:2d}", "[", f"{x[0]:+20.16f}", f"{x[1]:+20.16f}",\
           "]", f"{np.linalg.norm( x - x_, np.infty):4.1e}",\
          f"{np.linalg.norm( F(x), np.infty):4.1e}" )

print( "\nx = [", f"{x[0]:+20.16f}", "]"\
        "     x* = [", f"{x_[0]:+20.16f}", "]")
print( "    [", f"{x[1]:+20.16f}", "]"\
         "          [", f"{x_[1]:+20.16f}", "]")
print( "forward  error: || x - x* ||_oo = ", f"{np.linalg.norm(x-x_,np.infty):7.2e}",\
         "\nbackward error: || F(x) ||_oo   = ", f"{np.linalg.norm(F(x),np.infty):7.2e}")
print("\n")

|| F(x1) ||_oo = 8.88E-16
|| F(x2) ||_oo = 8.88E-16
 0 [  +1.0000000000000000  +1.0000000000000000 ] 2.0e+00 7.0e+00
 1 [  +8.0000000000000000  +4.0000000000000000 ] 5.2e+00 6.1e+01
 2 [  +9.0892018779342720 -12.6103286384976556 ] 1.2e+01 5.6e+02
 3 [  +0.2916638593780991  +3.7318816068896972 ] 4.7e+00 6.0e+01
 4 [  +1.0972908318748997  +5.6983105119746433 ] 6.6e+00 1.3e+02
 5 [  +0.0775153594730253  +1.7414997185988130 ] 2.7e+00 1.6e+01
 6 [  +0.3740092015758310  +0.7701322332852301 ] 2.4e+00 6.2e+00
 7 [  +2.2549401327459302  -2.1988498971524213 ] 1.2e+00 1.8e+01
 8 [ -104.9626363405546812 +217.1421389072866646 ] 2.2e+02 1.8e+05
 9 [  +3.0997663278467797  -1.8562303884551454 ] 9.1e-01 8.2e+00
10 [  +2.8335876498501165  -1.9850625420180319 ] 1.0e+00 1.2e+01
11 [  +2.9256371659376033  -1.9566792606157448 ] 1.0e+00 1.1e+01
12 [  +2.9320532029028756  -1.9703616045057897 ] 1.0e+00 1.1e+01
13 [  +3.1913218167443840  -3.1834306406349162 ] 2.2e+00 3.4e+01
14 [  +2.8114975032175837  -1.533535